Lê o dicionário features_padronizadas.csv (colunas: nome_original, nome_pt, dimensao, exemplo_valor);

Varre as pastas principais (as da sua imagem);

Para cada CSV encontrado:

Faz backup do original como *_old.csv (se ainda não existir);

Renomeia apenas as colunas que tiverem mapeamento para português (nome_pt);

Mantém colunas não mapeadas como estão (simples);

Salva de volta com o nome original;

Registra no inventário de auditoria uma linha contendo todas as nome_pt daquele arquivo.

In [1]:
# === Normalização de nomes de features para PT-BR (com backup simples) ===
# Requisitos: pandas, tqdm
# O que faz:
#  - Lê dicionário canônico "features_padronizadas.csv"
#  - Varre as pastas principais e processa todos .csv
#  - Para cada arquivo:
#      * cria BACKUP simples: <nome>_old.csv (se não existir)
#      * renomeia colunas que tiverem mapeamento nome_original -> nome_pt
#      * mantém demais colunas intactas
#      * salva com o NOME ORIGINAL
#  - Atualiza inventário: outputs/auditoria_normalizacao_features.csv
#
# Observação: não processa arquivos *_old.csv nem o próprio inventário.

import pandas as pd
from pathlib import Path
from tqdm import tqdm
import shutil
import hashlib
import datetime as dt

# ---------- CONFIG ----------
BASE_DIR = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
CSV_DIC  = BASE_DIR / "features_padronizadas.csv"

GLOBS = [
    BASE_DIR / "data" / "curated" / "*.csv",
    BASE_DIR / "outputs" / "*.csv",
    BASE_DIR / "outputs" / "baseline_datasets" / "*.csv",
    BASE_DIR / "outputs" / "pedra" / "*.csv",
    BASE_DIR / "outputs" / "eda" / "*.csv",
    BASE_DIR / "outputs" / "indices" / "*.csv",
]

OUT_DIR       = BASE_DIR / "outputs"
OUT_AUDITORIA = OUT_DIR / "auditoria_normalizacao_features.csv"

# ---------- FUNÇÕES AUX ----------
def sha256_file(p: Path) -> str:
    h = hashlib.sha256()
    with p.open("rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def norm_key(s: str) -> str:
    return (s or "").strip().lower()

def read_csv_header_any_encoding(p: Path):
    # Lê só o cabeçalho (nrows=0) tentando encodings comuns
    for enc in ("utf-8", "utf-8-sig", "latin1"):
        try:
            return pd.read_csv(p, nrows=0, encoding=enc), enc
        except Exception:
            continue
    # fallback: sem encoding explícito
    return pd.read_csv(p, nrows=0), None

def read_csv_any_encoding(p: Path):
    for enc in ("utf-8", "utf-8-sig", "latin1"):
        try:
            return pd.read_csv(p, encoding=enc, low_memory=False), enc
        except Exception:
            continue
    return pd.read_csv(p, low_memory=False), None

# ---------- CARREGAR DICIONÁRIO ----------
if not CSV_DIC.exists():
    raise FileNotFoundError(f"Dicionário não encontrado: {CSV_DIC}")

dic_hash = sha256_file(CSV_DIC)
df_dic = pd.read_csv(CSV_DIC, dtype=str).fillna("")
df_dic["nome_original"] = df_dic["nome_original"].map(norm_key)
df_dic["nome_pt"]       = df_dic["nome_pt"].astype(str).str.strip()
MAP_ORIG_TO_PT = dict(zip(df_dic["nome_original"], df_dic["nome_pt"]))

OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------- COLETAR ALVOS ----------
alvos = []
for g in GLOBS:
    alvos.extend(sorted(g.parent.glob(g.name)))

# Filtra CSVs e remove duplicados, *_old e o inventário
seen = set()
arquivos = []
for p in alvos:
    if p.suffix.lower() != ".csv": 
        continue
    if p.name.endswith("_old.csv"):
        continue
    if p.resolve() == OUT_AUDITORIA.resolve():
        continue
    key = str(p.resolve())
    if key not in seen and p.exists():
        seen.add(key)
        arquivos.append(p)

print(f"Dicionário: {CSV_DIC} | Itens: {len(MAP_ORIG_TO_PT)} | SHA256: {dic_hash}")
print(f"Arquivos a processar: {len(arquivos)}")

# ---------- PROCESSAMENTO ----------
registros = []
agora = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

for csv_path in tqdm(arquivos, desc="Normalizando features"):
    try:
        # Lê cabeçalho para decidir mapeamento
        df_head, enc_head = read_csv_header_any_encoding(csv_path)
        cols = list(df_head.columns)

        # Prepara novo nome (mapa parcial: só mapeia quem existir no dicionário)
        novo_nome = {}
        mapped_pt = []
        unmapped = []
        for c in cols:
            key = norm_key(c)
            if key in MAP_ORIG_TO_PT:
                pt = MAP_ORIG_TO_PT[key]
                novo_nome[c] = pt
                mapped_pt.append(pt)
            else:
                unmapped.append(c)

        # Se nada foi mapeado, apenas audita e segue
        if not novo_nome:
            registros.append({
                "arquivo_relativo": str(csv_path.relative_to(BASE_DIR)),
                "total_colunas": len(cols),
                "qtd_mapeadas": 0,
                "qtd_nao_mapeadas": len(cols),
                "todas_features_pt": "",
                "orig_nao_mapeadas": ";".join(unmapped),
                "hash_dicionario": dic_hash,
                "datahora_processo": agora,
                "acao": "somente_auditoria",
                "status": "ok"
            })
            continue

        # BACKUP simples (uma única vez)
        backup_path = csv_path.with_name(csv_path.stem + "_old.csv")
        if not backup_path.exists():
            shutil.copy2(csv_path, backup_path)

        # Lê tudo, renomeia parcialmente e salva de volta com NOME ORIGINAL
        df_full, enc_full = read_csv_any_encoding(csv_path)
        df_full_ren = df_full.rename(columns=novo_nome)
        df_full_ren.to_csv(csv_path, index=False, encoding=enc_full or "utf-8")

        registros.append({
            "arquivo_relativo": str(csv_path.relative_to(BASE_DIR)),
            "total_colunas": len(cols),
            "qtd_mapeadas": len(novo_nome),
            "qtd_nao_mapeadas": len(unmapped),
            # todas as features PT daquele arquivo (apenas as mapeadas)
            "todas_features_pt": ";".join(mapped_pt),
            "orig_nao_mapeadas": ";".join(unmapped),
            "hash_dicionario": dic_hash,
            "datahora_processo": agora,
            "acao": "backup_e_normalizacao",
            "status": "ok"
        })

    except Exception as e:
        registros.append({
            "arquivo_relativo": str(csv_path.relative_to(BASE_DIR)),
            "total_colunas": None,
            "qtd_mapeadas": None,
            "qtd_nao_mapeadas": None,
            "todas_features_pt": "",
            "orig_nao_mapeadas": "",
            "hash_dicionario": dic_hash,
            "datahora_processo": agora,
            "acao": "erro",
            "status": str(e)
        })

# ---------- INVENTÁRIO / AUDITORIA ----------
df_aud = pd.DataFrame(registros).sort_values(by=["arquivo_relativo"], kind="mergesort")
df_aud.to_csv(OUT_AUDITORIA, index=False, encoding="utf-8")

print("\nConcluído.")
print(f"Inventário: {OUT_AUDITORIA}")
print(df_aud[["arquivo_relativo","acao","qtd_mapeadas","qtd_nao_mapeadas"]].head(20).to_string(index=False))


Dicionário: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\features_padronizadas.csv | Itens: 150 | SHA256: 73078b08b9ab591a9871889bdcb0e69c04329ab83d069534ffd21dc19ef8a5df
Arquivos a processar: 26


Normalizando features: 100%|██████████| 26/26 [00:03<00:00,  6.59it/s]


Concluído.
Inventário: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\auditoria_normalizacao_features.csv
                                         arquivo_relativo                  acao  qtd_mapeadas  qtd_nao_mapeadas
                     data\curated\a1_physics_informed.csv backup_e_normalizacao           150                 0
            data\curated\a1_physics_informed_enriched.csv backup_e_normalizacao           150                 4
             data\curated\a1_physics_informed_proxies.csv backup_e_normalizacao           150                16
    outputs\baseline_datasets\compare_baseline_vs_off.csv     somente_auditoria             0                 4
   outputs\baseline_datasets\physics_baseline_proxies.csv backup_e_normalizacao           150                16
outputs\baseline_datasets\physics_offbaseline_proxies.csv backup_e_normalizacao           150                16
             outputs\baseline_datasets\stats_baseline.csv     somente_auditoria             0     

In [2]:
# === Expandir dicionário: coletar NÃO MAPEADAS do inventário e acrescentar ao features_padronizadas.csv ===
# O que faz:
#  1) Lê outputs/auditoria_normalizacao_features.csv para descobrir colunas 'orig_nao_mapeadas'
#  2) Deduplica e lista em quais arquivos cada coluna aparece
#  3) Gera "sugestao_nome_pt" com as MESMAS regras determinísticas anteriores
#  4) Coleta um exemplo de valor (primeiro não nulo) de alguma fonte onde a coluna exista
#  5) Salva propostas em outputs/propostas_novas_features.csv
#  6) Faz BACKUP simples do dicionário (features_padronizadas_old.csv) e ACRESCENTA as novas entradas no features_padronizadas.csv
#
# Observações:
#  - 'dimensao' será deixada em branco ("") para posterior preenchimento humano, evitando inferências arriscadas.
#  - Não altera nenhum arquivo de dados. Só lê quando precisa pegar exemplo de valor.

import pandas as pd
from pathlib import Path
from collections import defaultdict
import re, unicodedata
import shutil

# ============== CONFIGURAÇÃO ==============
BASE_DIR = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL")
ARQ_AUDITORIA = BASE_DIR / "outputs" / "auditoria_normalizacao_features.csv"
ARQ_DIC       = BASE_DIR / "features_padronizadas.csv"
ARQ_DIC_BKP   = BASE_DIR / "features_padronizadas_old.csv"
ARQ_PROPOSTAS = BASE_DIR / "outputs" / "propostas_novas_features.csv"

# Para buscar exemplo de valor, usaremos estes diretórios (mesmos da auditoria):
BUSCA_DIRS = [
    BASE_DIR / "data" / "curated",
    BASE_DIR / "outputs",
    BASE_DIR / "outputs" / "baseline_datasets",
    BASE_DIR / "outputs" / "pedra",
    BASE_DIR / "outputs" / "eda",
    BASE_DIR / "outputs" / "indices",
]

# ============== UTILITÁRIOS ==============
def norm_key(s: str) -> str:
    return (s or "").strip().lower()

def read_csv_any_encoding(p: Path, nrows=None):
    for enc in ("utf-8", "utf-8-sig", "latin1"):
        try:
            return pd.read_csv(p, encoding=enc, nrows=nrows, low_memory=False), enc
        except Exception:
            continue
    return pd.read_csv(p, nrows=nrows, low_memory=False), None

def strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

def snake_clean(s: str) -> str:
    s = re.sub(r"[^a-z0-9_]+", "_", s.lower())
    s = re.sub(r"_+", "_", s).strip("_")
    return s

# === Regras determinísticas (iguais às usadas antes) ===
phrase_rules = [
    (r"total_paf_air_flow",        "vazao_ar_prim_total"),
    (r"total_air_flow",            "vazao_ar_total"),
    (r"main_steam_flow_sel",       "vazao_vapor_principal"),
    (r"main_steam_pressure",       "pressao_vapor_principal"),
    (r"hot_reheat_steam_pressure", "pressao_vapor_reaquec"),
    (r"hot_reheat_steam_temperature", "temp_vapor_reaquec"),
    (r"drum_level_sel",            "nivel_tambor"),
    (r"drum_pres_sel",             "pressao_tambor"),
    (r"furnace_pres",              "pressao_fornalha"),
    (r"feed_water_flw_sel",        "vazao_agua_alimentacao"),
    (r"o2_avg_?$",                 "o2_medio"),
    (r"status_operacao",           "status_operacao"),
    (r"flw_total_a",               "vazao_total_a"),
    (r"flw_total_b",               "vazao_total_b"),
    (r"flw_total_c",               "vazao_total_c"),
]

token_map = {
    "cur": "corr", "current": "corrente",
    "prs": "pressao", "press": "pressao", "pressure": "pressao",
    "temp": "temp", "temperature": "temp", "te": "temp",
    "flw": "vazao", "flow": "vazao",
    "level": "nivel", "avg": "medio",

    "fan": "vent", "mtr": "motor",
    "idf": "vent_tiragem", "id": "vent_tiragem",
    "pa": "ar_prim", "sa": "ar_sec",
    "hpfa": "ar_flu_ap", "hpa": "ar_alta_press",
    "aph": "preaq_ar", "ltr": "ltr", "ecmz": "econ",
    "deaerator": "desaerador", "dea": "desaerador",
    "condensate": "condensado", "pump": "bomba",
    "furnace": "fornalha", "bed": "leito", "drum": "tambor",
    "steam": "vapor", "water": "agua", "gas": "gas",

    "inlet": "entrada", "inl": "entrada",
    "outlet": "saida",  "outl": "saida",
    "hdr": "coletor", "pos": "pos", "fdb": "fbk",

    "upper": "sup", "lower": "inf", "medium": "medio",
    "side": "lado", "a": "a", "b": "b", "c": "c",

    "o2": "o2", "so2": "so2", "dust": "poeira", "coal": "carvao",

    "of": "", "with": "", "and": "", "sel": "", "ol": "", "mi": ""
}

def apply_phrase_rules(s: str) -> str:
    out = s
    for pat, rep in phrase_rules:
        out = re.sub(pat, rep, out)
    return out

def translate_tokens(name: str) -> str:
    # remover sufixos de unidade recorrentes no NOME (dimensão fica a cargo humano depois)
    name = re.sub(r"_(t_h|adegc|mpa|kpa|nm3_h|m3_s|rpm|mm|mw|pa)$", "", name)
    name = apply_phrase_rules(name)
    tokens = [t for t in re.split(r"_+", name) if t]
    out = []
    for t in tokens:
        rep = token_map.get(t, t)
        for subt in rep.split("_"):
            if subt:
                out.append(subt)
    compact = []
    for tok in out:
        if not compact or compact[-1] != tok:
            compact.append(tok)
    candidate = "_".join(compact) if compact else "var"
    candidate = strip_accents(candidate)
    candidate = snake_clean(candidate)
    return candidate

# ============== 1) Ler inventário e coletar NÃO MAPEADAS ==============
if not ARQ_AUDITORIA.exists():
    raise FileNotFoundError(f"Inventário de auditoria não encontrado: {ARQ_AUDITORIA}")
if not ARQ_DIC.exists():
    raise FileNotFoundError(f"Dicionário não encontrado: {ARQ_DIC}")

aud = pd.read_csv(ARQ_AUDITORIA, dtype=str).fillna("")

# Mapa: coluna_nao_mapeada -> {arquivos onde aparece}
unmapped_sources = defaultdict(set)
for _, row in aud.iterrows():
    arq = row.get("arquivo_relativo", "")
    raw = row.get("orig_nao_mapeadas", "")
    if not raw:
        continue
    for col in [c.strip() for c in raw.split(";") if c.strip()]:
        unmapped_sources[col].add(arq)

lista_unmapped = sorted(unmapped_sources.keys(), key=lambda x: x.lower())
print(f"NÃO mapeadas distintas encontradas: {len(lista_unmapped)}")

if not lista_unmapped:
    print("Nada a fazer: não há colunas não mapeadas no inventário.")
    # encerra graciosamente
    raise SystemExit(0)

# ============== 2) Carregar dicionário atual para checar conflitos ==============
dic = pd.read_csv(ARQ_DIC, dtype=str).fillna("")
dic["nome_original"] = dic["nome_original"].map(norm_key)
dic["nome_pt"]       = dic["nome_pt"].astype(str).str.strip()

exist_pt = set(dic["nome_pt"].tolist())
exist_orig = set(dic["nome_original"].tolist())

# ============== 3) Propor nome_pt e coletar exemplo de valor ==============
def achar_exemplo_valor(col_original: str, fontes_relativas: set[str]):
    """
    Tenta encontrar um exemplo de valor para 'col_original' procurando nas fontes informadas.
    Leitura limitada (nrows=None mas early-stop ao achar um não-nulo).
    Match de coluna é case-insensitive e ignora espaços.
    """
    alvo_norm = norm_key(col_original)
    # lista de caminhos absolutos candidatos
    candidatos = []
    for rel in fontes_relativas:
        p = BASE_DIR / Path(rel)
        if p.exists() and p.suffix.lower()==".csv":
            candidatos.append(p)

    # fallback: procura pelo nome do arquivo em diretórios padrão se caminho relativo não existir
    if not candidatos:
        for d in BUSCA_DIRS:
            for p in d.glob("*.csv"):
                # heurística simples: se o arquivo contém a coluna no cabeçalho (nrows=0)
                try:
                    df_head, _ = read_csv_any_encoding(p, nrows=0)
                    cols_norm = [norm_key(c) for c in df_head.columns]
                    if alvo_norm in cols_norm:
                        candidatos.append(p)
                except Exception:
                    continue

    for p in candidatos:
        try:
            df, _ = read_csv_any_encoding(p)
            # localizar a coluna real por normalização
            col_match = None
            mapa = {norm_key(c): c for c in df.columns}
            if alvo_norm in mapa:
                col_match = mapa[alvo_norm]
            if col_match is None:
                continue
            serie = df[col_match].dropna()
            if not serie.empty:
                # pega o primeiro não-nulo como string curta
                val = str(serie.iloc[0])
                return val[:120]
        except Exception:
            continue
    return ""

propostas = []
seen_sugestoes = set(exist_pt)  # para garantir unicidade vs dicionário atual

for orig in lista_unmapped:
    orig_norm = norm_key(orig)
    if orig_norm in exist_orig:
        # Já existe no dicionário (raro, mas defensivo)
        continue

    sug = translate_tokens(orig_norm)
    base = sug
    # garantir unicidade com sufixo _2, _3 ... se colidir com existentes
    i = 2
    while sug in seen_sugestoes:
        sug = f"{base}_{i}"
        i += 1
    seen_sugestoes.add(sug)

    exemplo = achar_exemplo_valor(orig, unmapped_sources[orig])
    propostas.append({
        "nome_original": orig_norm,
        "sugestao_nome_pt": sug,
        "fontes": ";".join(sorted(unmapped_sources[orig])),
        "exemplo_valor": exemplo
    })

df_prop = pd.DataFrame(propostas)
ARQ_PROPOSTAS.parent.mkdir(parents=True, exist_ok=True)
df_prop.to_csv(ARQ_PROPOSTAS, index=False, encoding="utf-8")
print(f"Propostas salvas em: {ARQ_PROPOSTAS}  |  Novas entradas: {len(df_prop)}")

# ============== 4) Acrescentar ao dicionário (backup simples) ==============
if not df_prop.empty:
    # backup simples, uma única vez (não sobrescreve se já existir)
    if not ARQ_DIC_BKP.exists():
        shutil.copy2(ARQ_DIC, ARQ_DIC_BKP)
        print(f"Backup do dicionário criado: {ARQ_DIC_BKP}")

    df_new = pd.DataFrame({
        "nome_original": df_prop["nome_original"],
        "nome_pt": df_prop["sugestao_nome_pt"],
        "dimensao": ["" for _ in range(len(df_prop))],       # deixar em branco para preenchimento humano
        "exemplo_valor": df_prop["exemplo_valor"].fillna("")
    })

    # anexar ao final, preservando as colunas na ordem correta
    dic_cols = ["nome_original", "nome_pt", "dimensao", "exemplo_valor"]
    df_out = pd.concat([dic[dic_cols], df_new[dic_cols]], ignore_index=True)
    df_out.to_csv(ARQ_DIC, index=False, encoding="utf-8")
    print(f"Dicionário atualizado: {ARQ_DIC}  |  Linhas totais: {len(df_out)}")
else:
    print("Nenhuma proposta gerada; dicionário não foi modificado.")


NÃO mapeadas distintas encontradas: 84
Propostas salvas em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\outputs\propostas_novas_features.csv  |  Novas entradas: 84
Backup do dicionário criado: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\features_padronizadas_old.csv
Dicionário atualizado: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\features_padronizadas.csv  |  Linhas totais: 234
